# Assignments - module 1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec/blob/main/assignments/Assignments_1.ipynb)

This notebook contains the assignments to complete for credits for the first module. 

**Submission**: Once you're happy with your solutions, send it to me in any form (email the file, share it through Colab/Google Drive, send me a link to your GitHub repo...).

**Deadline**: 15th of July 2023

**Evaluation**: There is no grade, but I will pass assignments that showcase a reasonable degree of understanding og the covered topics. Do your best, and feel free to ask for help if you are struggling! 

(Also, try to keep in mind not only the goal of the exercise, but also all the coding best practices we have been considering in the lectures.)

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## 0. Spike detection

In this exercise we will be playing around with some (dummy) electrophysiology recordings. Let's start by having a look at the raw data!

In [ ]:
def generate_spike_trace(trace_length=60, firing_rate=1, noise_sigma = 0.05):
    """Function to generate a fake extracellular recording.
    
    Parameters
    ----------
        trace_length : float
            Duration of the recording in seconds.
        
        firing_rate : float
            Average firing rate of the neuron in Hz.
            
        noise_sigma : float
            Noise level.
            
            
    Returns:
    --------
        np.array
            Fake extracellular recording. 
    
    """
    np.random.seed(42)
    FS = 10000  # sampling frequency
    n = int(trace_length * FS)  # number of samples
    
    # Generate spike shape template as a difference of Gaussians.
    # A horrible bunch of magic numbers - do not imitate!
    x = np.arange(30)
    spike_template = np.exp(-(x - 10)**2/6) - np.exp(-(x - 12)**2/16)*0.8

    # Generate spike times from a gaussian distribution:
    spikes_times = np.random.poisson(firing_rate / FS, n)
    
    # Convolve dirac delta functions of spike times with spike template:
    trace = np.convolve(spikes_times, spike_template)[:n]

    # Add some gaussian noise:
    trace += np.random.normal(0, noise_sigma, n)
    
    return trace

### Exercise 0.0

Run the function below to generate an synthetic extracellular recording for a neuron. Make a nice plot with the trace; the spikes are the peaks appearing above the noise! 

---

(_Optional_) If you want to make a plot with exact x coordinates in seconds, you should know that the trace is sampled at 10000 Hz (10000 points per second).

### Exercise 0.1

Write a function to detect spikes!
The function should take the trace as input, and return the index of each spike as the output (as the index, you should take the position of the spike maximum)

Hint: a good strategy to detect such events is to set a threshold, and look for elements above it. This will not be enough! each spike could have more than 1 point above the threshold, but you want to make sure you take only the spike peak! For this, you will probably need a loop.

Hint: do not start from writing the function. First debug your code running it in a cell, then move it to a function.

Hint: if you want, you can quickly check out the results you are getting by making a scatter plot of the detected spikes overimposed on the electrophysiology trace! (use as x of the dots the indexes of the spikes, and as y the hight of the trace at those indexes)

### Exercise 0.2

We now want to have a look at the shape of those spikes. For this, we will create a function that crops small chunks of the trace around each spike peak.

Write a `crop_event()` function that takes as inputs:
   - the recording array
   - the spike indexes
   - a `n_points_pad` variable specifying the number of points to include before and after the spike

And returns a matrix of shape `(n_spikes, n_points*2)` containing the trace chunks cropped around spike events! 

Hint: A good strategy coult be initialize an empty matrix and then fill it in a loop with the trace around the spikes.

**This function can be very useful in many contexts!** You can use it every time you want to crop a timeseries around events (e.g., EEG data or video kinematics data around some stimuli). So keep it at hand in the future!

---

(_Optional_) Pro challenge: Try to do it without for loops! if you construct a matrix with the indexes of the points you want to exctract from the trace, you can use it directly to index the trace!
For indexing in this way, you want to build a matrix that looks like this:
```
array([[...t0-2, t0-1, t0, t0+1, t0+2...],
       [...t1-2, t1-1, t1, t1+1, t1+2...],
       [...t2-2, t2-1, t2, t2+1, t2+2...],])
```
Where `t0`, `t1`, `t2`... are the indexes of each spike, and you take as many points before and after as specified by the `n_points_pad` paramenter. 

Building this matrix without loops is not trivial but it can be done nicely with numpy broadcasting!

### Exercise 0.3

Finally, make two subplots one close to the other. On the left, use `plt.matshow` to show the spike matrix. On the right,
plot each individual spike (rows of the matrix) using `plt.plot` with gray lines, and the average spike shape in red on top.

---

(Optional) If you want you can try to normalize the matrix before plotting by subtracting the average of each row (as we were doing for the daily temperatures)!

## 1. Real books data

After having appreciated how many books the universe of all possible books contains, let's now focus just on the reachable ones - and how much people like them! 

Here, we will download the information about about thousands volumns available on Amazon. Just a tiny fraction of Babel's books, but way more organized!

We will also get a dataset of users writing reviews, and a dataset of reviews.

### Exercise 1.0

Using, `pandas`, read the `.csv` files containing the books, the ratings, and the user data that you can find at the  urls defined below.

Then, plot an histogram of all the ratings from all users, and another histogram with the age of the users:

In [ ]:
users_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/users.csv"
ratings_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/ratings.csv"
books_df_url = "https://github.com/vigji/python-cimec/raw/main/assignments/files/books.csv"


### Exercise 1.1

Using the ratings dataframe, compute the average rating for each book, and count how many reviews each book had. Then:
- find out which book had the highest number of reviews.
- find out which book had the highest average rating - but include only books that have at least 100 reviews!


Finally, look for the titles that correspond to those book codes (ISBNs are unique book codes).

### Exercise 1.2

Let's get even more specific! Let's find the preferences of users in specific countries and with different ages.

Use the users DataFrame to select only italian users under 40 years old. Then, go back to the reviews dataframe and filter only reviews from those users. Compute the average ratings (include only books that have at least 3 reviews) and sort the ISBNs by average rating. Finally, find the books corresponding to each ISBN code to get which books got the best ratings in this coort of people!

(_Optional_): from the users DataFrame generate a list of all the countries present in the dataset. Then, find the highest rated book in each one of those countries.